# An Introduction to Process Mining :Trade Payables Use Case

Most companies have information systems that record activities of interests, such as the registration of a new customer, the sale of a product, the approval of a purchase system, the processing of a payment system, etc. All of these activities result in one or more events being recorded in some information system. These events are usually used for record-keeping, accounting, auditing, etc.

Process mining is concerned with using these recorded activities in order to understand how an organisation works. Using process mining, actual sequence of tasks (events) that are performed can be automatically discovered, revealing the behaviour of the recorded process execution. It is therefore possible to compare the actual process with the expected behaviour and deviations can be detected. This can lead to identification of process diagnostics and preventive action for potential risks and fraud. To learn more about process mining, visit XXXXXX.

Trade receivables are obligations by a company to pay for goods or services that have been acquired from suppliers in the ordinary course of business. Purchase-to-pay process is recognised as one of the most important processes within a company because it provides core resources for running a business on a daily basis and strongly influences overall costs and timing of production. It starts with filing a purchase order/request and is completed when the final payment is made to the vendor.

In a procurement process, there are different risks inherent such as; fictious transactions being recorded, payment has been made without an underlying purchase, purchases not properly authorised, etc. Therefore, when auditing trade payables, it is very important to understand how the purchase-to-pay transactions are processed, and the controls available in the process, such as segregation of duties.

In this post, we will look at how process mining can be used to understand the purchase-to-pay process of a company, who is responsible for carrying out which tasks and how the tasks are handed over from one employee to another. This post is an introduction to the possibile use case of process mining in auditing. Further posts will touch on more use cases even outside the field of auditing.

This will be done using python and various libraries such as pandas (for analysing the data) and graphviz (for drawing the directly followed graph showing the process).

The dataset was gotten from https://github.com/IBM/processmining. IBM Github repository for process mining.


The event log is fully IEEE-XES compliant and is structured as follows. The case ID is a combination of the purchase document and the purchase item. There is a total of 76,349 purchase documents containing in total 251,734 items, i.e. there are 251,734 cases. In these cases, there are 1,595,923 events relating to 42 activities performed by 627 users (607 human users and 20 batch users). Sometimes the user field is empty, or NONE, which indicates no user was recorded in the source system.

For each purchase item (or case) the following attributes are recorded:

- 1 Key: The purchase ID,
- 2 Date: The date and time of an event,
- 3 User: The user resource involved in the process,
- 4 Activity: The activity performed in the process,
- 5 Product_hierarchy: A text explaining the hierachy of a purchase item,
- 6 NetValue: The value of a purchase item,
- 7 Delivery: The delivery ID of this item,
- 8 Delivery_Date: The delivery date of this item,
- 9 Good_Issue_Date: The date goods was issued. However this was derived,
- 10 Difference: The time difference (in seconds) between the delivery date and goods issue date,
- 11 Customer: The customer id,
- 12 OrderType: Type of order,
- 13 clientCode: The client code,
- 14 NotInTime: Indicating if an order was delayed or not where 1 = delayed and 0 = on time,
- 15 Execution_Status: Indicating if it was a manual or automatic task,
- 16 User_Type: Indicating if the task was done by a human or robot,
- 17 Change_Status: Change indicator,
- 18 ID_Change_Status: The change_status ID,
- 19 Block_Status: Block indicator,
- 20 ID_Block_Status: The Block_Status ID,


In [1]:
import pandas as pd
import numpy as np
import graphviz
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
df = pd.read_csv("o2c_crypted.csv", thousands='.', decimal=',')
df

C:\Users\CodeVader\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Key,Date,User,Activity,Role,Product_hierarchy,NetValue,Company,Delivery,Delivery_Date,...,Delayed,PromiseMAD,ActualMAD,Execution_Status,User_Type,Change_Status,ID_Change_Status,Block_Status,ID_Block_Status,Local_Family_code
0,7020029102_10,04/01/2016 13:46:13,User1,Line Creation,Customer Service Representative,TLC Optical Cables,773.87,767,NaN,NaN,...,IN TIME,1.452726e+12,1.452726e+12,Manual,Human,no change,With change,no block,With block,LocalFamily1
1,7020029103_10,04/01/2016 13:46:55,User1,Line Creation,Customer Service Representative,TLC Optical Cables,706.50,767,NaN,NaN,...,IN TIME,1.482102e+12,1.452812e+12,Manual,Human,no change,With change,no block,With block,LocalFamily2
2,7020029104_10,04/01/2016 13:47:30,User1,Line Creation,Customer Service Representative,TLC Optical Cables,2168.40,767,NaN,NaN,...,IN TIME,1.453417e+12,1.453417e+12,Manual,Human,no change,With change,no block,With block,LocalFamily2
3,7020029104_20,04/01/2016 13:47:38,User1,Line Creation,Customer Service Representative,TLC Optical Cables,1566.60,767,NaN,NaN,...,IN TIME,1.453417e+12,1.453417e+12,Manual,Human,no change,With change,no block,With block,LocalFamily3
4,7020029104_30,04/01/2016 13:47:43,User1,Line Creation,Customer Service Representative,TLC Optical Cables,1106.85,767,NaN,NaN,...,IN TIME,1.482102e+12,1.453417e+12,Manual,Human,no change,With change,no block,With block,LocalFamily2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251473,7020034883_90,11/07/2017 23:59:59,User66,Good Issue,Customer Service Representative,TLC Optical Cables,59994.48,767,7.050145e+09,02/08/17,...,IN TIME,1.500930e+12,1.500590e+12,Manual,Human,no change,With change,no block,With block,LocalFamily9
251474,7020034883_90,11/07/2017 23:59:59,User66,Good Issue,Customer Service Representative,TLC Optical Cables,59994.48,767,7.050145e+09,02/08/17,...,IN TIME,1.500930e+12,1.500590e+12,Manual,Human,no change,With change,no block,With block,LocalFamily9
251475,7020030338_150,12/07/2017 23:59:59,User66,Good Issue,Customer Service Representative,TLC Optical Cables,89136.00,767,7.050145e+09,07/05/18,...,IN TIME,1.525040e+12,1.501020e+12,Manual,Human,no change,With change,no block,With block,LocalFamily13
251476,7020033072_100,13/07/2017 23:59:59,User66,Good Issue,Customer Service Representative,TLC Optical Cables,17013.15,767,7.050145e+09,01/08/17,...,IN TIME,1.500930e+12,1.500930e+12,Manual,Human,no change,With change,no block,With block,LocalFamily8


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251478 entries, 0 to 251477
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Key                251478 non-null  object 
 1   Date               251478 non-null  object 
 2   User               251478 non-null  object 
 3   Activity           251478 non-null  object 
 4   Role               251478 non-null  object 
 5   Product_hierarchy  251478 non-null  object 
 6   NetValue           251478 non-null  float64
 7   Company            251478 non-null  int64  
 8   Delivery           110283 non-null  float64
 9   Delivery_Date      110276 non-null  object 
 10  Good_Issue_Date    251478 non-null  float64
 11  Difference         110283 non-null  float64
 12  Customer           251478 non-null  object 
 13  OrderType          251478 non-null  object 
 14  clientCode         251478 non-null  object 
 15  NotInTime          251478 non-null  int64  
 16  De

From the above, it is aparrent that the date column is not recognised as a date. 

In [4]:
df['Date'] = pd.to_datetime(df['Date'],  dayfirst=True)
df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'],  dayfirst=True)
df = df.sort_values(by=['Key', 'Date'], ascending=True).reset_index(drop=True)
df

,Key,Date,User,Activity,Role,Product_hierarchy,NetValue,Company,Delivery,Delivery_Date,...,Delayed,PromiseMAD,ActualMAD,Execution_Status,User_Type,Change_Status,ID_Change_Status,Block_Status,ID_Block_Status,Local_Family_code
0,0009000481_10,2016-01-05 17:10:00,User9,Line Creation,Customer Service Representative,TLC Optical Fibres,6920.00,767,NaN,NaT,...,IN TIME,0.0,1.452035e+12,Manual,Human,no change,Without change,no block,With block,NaN
1,0009000481_10,2016-01-06 16:07:53,User9,LgstCheckOnConfDat Removed,Customer Service Representative,TLC Optical Fibres,6920.00,767,NaN,NaT,...,IN TIME,0.0,1.452035e+12,Manual,Human,no change,Without change,block,With block,NaN
2,0009000481_10,2016-01-06 18:05:04,User61,Delivery,System Automatic Job,TLC Optical Fibres,6920.00,767,7.070100e+09,2016-01-08,...,IN TIME,0.0,1.452030e+12,Automatic,Robot,no change,Without change,no block,With block,NaN
3,0009000481_10,2016-01-07 23:59:59,User61,Good Issue,System Automatic Job,TLC Optical Fibres,6920.00,767,7.070100e+09,2016-01-08,...,IN TIME,0.0,1.452030e+12,Automatic,Robot,no change,Without change,no block,With block,NaN
4,0009000485_10,2016-01-14 19:07:11,User16,Line Creation,NA Fiber Sales and Service Manager,TLC Optical Fibres,735.25,767,NaN,NaT,...,IN TIME,0.0,1.452726e+12,Manual,Human,no change,Without change,no block,With block,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251473,7080001018_40,2017-07-07 17:08:47,User22,Header Block Removed,Customer Service Representative,TLC Optical Cables,4725.71,767,NaN,NaT,...,DELAYED,0.0,1.499378e+12,Manual,Human,no change,Without change,block,With block,LocalFamily13
251474,7080001018_40,2017-07-11 15:06:56,User60,Delivery,System Automatic Job,TLC Optical Cables,4725.71,767,7.090001e+09,2017-07-07,...,DELAYED,0.0,1.499380e+12,Automatic,Robot,no change,Without change,no block,With block,LocalFamily13
251475,7080001018_40,2017-07-13 23:59:59,User61,Good Issue,System Automatic Job,TLC Optical Cables,4725.71,767,7.090001e+09,2017-07-07,...,DELAYED,0.0,1.499380e+12,Automatic,Robot,no change,Without change,no block,With block,LocalFamily13
251476,7080001026_10,2017-07-13 19:38:08,User5,Line Creation,Customer Service Representative,TLC Optical Cables,4990.43,767,NaN,NaT,...,IN TIME,0.0,1.499897e+12,Manual,Human,no change,Without change,no block,With block,LocalFamily47


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251478 entries, 0 to 251477
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Key                251478 non-null  object        
 1   Date               251478 non-null  datetime64[ns]
 2   User               251478 non-null  object        
 3   Activity           251478 non-null  object        
 4   Role               251478 non-null  object        
 5   Product_hierarchy  251478 non-null  object        
 6   NetValue           251478 non-null  float64       
 7   Company            251478 non-null  int64         
 8   Delivery           110283 non-null  float64       
 9   Delivery_Date      110276 non-null  datetime64[ns]
 10  Good_Issue_Date    251478 non-null  float64       
 11  Difference         110283 non-null  float64       
 12  Customer           251478 non-null  object        
 13  OrderType          251478 non-null  object  

In [9]:
df['Key'].nunique()

45825

In [8]:
df['Date'].aggregate(['min','max'])

min   2016-01-04 13:46:13
max   2017-07-16 16:06:01
Name: Date, dtype: datetime64[ns]

In [10]:
df['Activity'].value_counts()

Delivery                            55215
Good Issue                          55068
Line Creation                       45825
LgstCheckOnConfDat Removed          40292
Header Block Removed                25681
Sched.Line Changed Delivery Date    14287
Document released for credit         4820
Schedule Line Rejected               3182
Address missing Block Set            1412
Address missing Block Removed        1380
LgstCheckOnConfDat Set               1296
Sched.Line Block Removed              972
CTR Block Removed                     943
Document blocked for credit           638
Header Block Set                      451
Sched.Line Block Set                    6
Special test Block Set                  5
Special test Block Removed              4
CTR Block Set                           1
Name: Activity, dtype: int64

In [13]:
df['User'].value_counts()

User9     31810
User61    29027
User60    25702
User43    19026
User20    13565
          ...  
User50        2
User52        1
User40        1
User46        1
User53        1
Name: User, Length: 76, dtype: int64

The dataset comprise of 45825 order request (cases) and 251478 events.
Let us start by looking at the process flow

In [15]:
process_flow = df.copy()
process_flow = process_flow[['Key','Date','User','Activity']]
process_flow

,Key,Date,User,Activity
0,0009000481_10,2016-01-05 17:10:00,User9,Line Creation
1,0009000481_10,2016-01-06 16:07:53,User9,LgstCheckOnConfDat Removed
2,0009000481_10,2016-01-06 18:05:04,User61,Delivery
3,0009000481_10,2016-01-07 23:59:59,User61,Good Issue
4,0009000485_10,2016-01-14 19:07:11,User16,Line Creation
...,...,...,...,...
251473,7080001018_40,2017-07-07 17:08:47,User22,Header Block Removed
251474,7080001018_40,2017-07-11 15:06:56,User60,Delivery
251475,7080001018_40,2017-07-13 23:59:59,User61,Good Issue
251476,7080001026_10,2017-07-13 19:38:08,User5,Line Creation


In [16]:
process_flow['Count'] = process_flow.groupby('Key').cumcount()+1
process_flow

,Key,Date,User,Activity,Count
0,0009000481_10,2016-01-05 17:10:00,User9,Line Creation,1
1,0009000481_10,2016-01-06 16:07:53,User9,LgstCheckOnConfDat Removed,2
2,0009000481_10,2016-01-06 18:05:04,User61,Delivery,3
3,0009000481_10,2016-01-07 23:59:59,User61,Good Issue,4
4,0009000485_10,2016-01-14 19:07:11,User16,Line Creation,1
...,...,...,...,...,...
251473,7080001018_40,2017-07-07 17:08:47,User22,Header Block Removed,2
251474,7080001018_40,2017-07-11 15:06:56,User60,Delivery,3
251475,7080001018_40,2017-07-13 23:59:59,User61,Good Issue,4
251476,7080001026_10,2017-07-13 19:38:08,User5,Line Creation,1


In [17]:
activities_list = process_flow.copy()
activities_list = activities_list.pivot(index='Key', columns='Count', values='Activity')
# activities_list_full

# Get the last valid index (last activity per case)
def func(x):
    if x.last_valid_index() is None:
        return np.nan
    else:
        return x[x.last_valid_index()]
    
activities_list['Last Activity'] = activities_list.apply(func, axis=1)
activities_list['First Activity'] = activities_list[1]
activities_list

Count,1,2,3,4,5,6,7,8,9,10,...,382,383,384,385,386,387,388,389,Last Activity,First Activity
Key,,,,,,,,,,,,,,,,,,,,,
0009000481_10,Line Creation,LgstCheckOnConfDat Removed,Delivery,Good Issue,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Good Issue,Line Creation
0009000485_10,Line Creation,LgstCheckOnConfDat Set,LgstCheckOnConfDat Removed,Delivery,Good Issue,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Good Issue,Line Creation
0009000486_10,Line Creation,LgstCheckOnConfDat Removed,Delivery,Good Issue,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Good Issue,Line Creation
0009000487_10,Line Creation,LgstCheckOnConfDat Removed,Delivery,Good Issue,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Good Issue,Line Creation
0009000487_100,Line Creation,LgstCheckOnConfDat Removed,Delivery,Good Issue,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Good Issue,Line Creation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7080001011_20,Line Creation,Header Block Removed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Header Block Removed,Line Creation
7080001014_30,Line Creation,Header Block Removed,Delivery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Delivery,Line Creation
7080001017_20,Line Creation,Header Block Removed,Delivery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Delivery,Line Creation


In [18]:
activities_list['Last Activity'].value_counts()

Good Issue                          36383
Line Creation                        3377
Schedule Line Rejected               3182
LgstCheckOnConfDat Removed           1453
Delivery                              632
Sched.Line Changed Delivery Date      283
Document released for credit          164
Header Block Removed                  131
Address missing Block Removed         118
Address missing Block Set              50
CTR Block Removed                      48
LgstCheckOnConfDat Set                  2
Header Block Set                        2
Name: Last Activity, dtype: int64

In [19]:
(36383/45825)*100

79.39552645935625

In [20]:
activities_list['First Activity'].value_counts()

Line Creation                       42309
Header Block Removed                 3262
Document released for credit          238
LgstCheckOnConfDat Removed             13
Sched.Line Changed Delivery Date        2
Address missing Block Removed           1
Name: First Activity, dtype: int64

In order for us to gain relevant information of the processes, it is important that we use those cases which are completed. To this end, we will fiter out those cases which do not end with "Good Issue". After this filter, we still have 36383 cases representing approximately 79% of our dataset. We believe this still is a great sample

In [26]:
xxx1 = activities_list.reset_index()
# xxx1
xxx1 = xxx1[xxx1['Last Activity'] == 'Good Issue'].reset_index(drop=True)
xxx1 = xxx1[['Key','Last Activity']]
xxx1

Count,Key,Last Activity
0,0009000481_10,Good Issue
1,0009000485_10,Good Issue
2,0009000486_10,Good Issue
3,0009000487_10,Good Issue
4,0009000487_100,Good Issue
...,...,...
36378,7080000979_94,Good Issue
36379,7080000990_390,Good Issue
36380,7080000997_10,Good Issue
36381,7080001003_10,Good Issue


In [30]:
process_flow_2 = process_flow.merge(xxx1, on="Key", how='left').dropna(axis=0).reset_index(drop=True)
process_flow_2

,Key,Date,User,Activity,Count,Last Activity
0,0009000481_10,2016-01-05 17:10:00,User9,Line Creation,1,Good Issue
1,0009000481_10,2016-01-06 16:07:53,User9,LgstCheckOnConfDat Removed,2,Good Issue
2,0009000481_10,2016-01-06 18:05:04,User61,Delivery,3,Good Issue
3,0009000481_10,2016-01-07 23:59:59,User61,Good Issue,4,Good Issue
4,0009000485_10,2016-01-14 19:07:11,User16,Line Creation,1,Good Issue
...,...,...,...,...,...,...
221011,7080001003_10,2017-06-22 23:59:59,User60,Good Issue,4,Good Issue
221012,7080001018_40,2017-07-07 15:37:43,User22,Line Creation,1,Good Issue
221013,7080001018_40,2017-07-07 17:08:47,User22,Header Block Removed,2,Good Issue
221014,7080001018_40,2017-07-11 15:06:56,User60,Delivery,3,Good Issue


In [31]:
process_flow_2['Key'].nunique()

36383